# IPL Auction Data - All Sheets Combined
### Output: player_name | team_name | Auction_Info | ipl_year

In [ ]:
import pandas as pd

FILE_PATH = '/mnt/user-data/uploads/auction_price_data.xlsx'

xl = pd.ExcelFile(FILE_PATH)
print(f"Sheets found: {xl.sheet_names}")

In [ ]:
# All known IPL teams (sorted longest-first to avoid partial matches)
IPL_TEAMS = sorted([
    'Chennai Super Kings',
    'Mumbai Indians',
    'Royal Challengers Bangalore',
    'Royal Challengers Bengaluru',
    'Kolkata Knight Riders',
    'Delhi Daredevils',
    'Delhi Capitals',
    'Kings XI Punjab',
    'Punjab Kings',
    'Rajasthan Royals',
    'Sunrisers Hyderabad',
    'Rising Pune Supergiant',
    'Rising Pune Supergiants',
    'Gujarat Lions',
    'Lucknow Super Giants',
    'Gujarat Titans'
], key=len, reverse=True)


def split_player_team(player_val):
    """
    Sold players  → Player column = player_name + team_name (concatenated)
    Unsold players → Player column = player_name only
    """
    player_str = str(player_val)
    for team in IPL_TEAMS:
        if team.lower() in player_str.lower():
            idx = player_str.lower().find(team.lower())
            return player_str[:idx].strip(), team
    return player_str.strip(), 'Unsold'


def process_sheet(sheet_name):
    df = pd.read_excel(FILE_PATH, sheet_name=sheet_name)
    df[['player_name', 'team_name']] = df['Player'].apply(
        lambda x: pd.Series(split_player_team(x))
    )
    df = df[['player_name', 'team_name', 'Amount', 'Auction Info']].copy()
    df.columns = ['player_name', 'team_name', 'amount', 'Auction_Info']
    df['ipl_year'] = sheet_name
    df['Retained'] = df['Auction_Info'].str.contains('retained', case=False, na=False).map({True: 'Yes', False: 'No'})
    return df


# Process all sheets and combine
all_dfs = [process_sheet(sheet) for sheet in xl.sheet_names]
final_df = pd.concat(all_dfs, ignore_index=True)

print(f"Total records across all sheets: {len(final_df)}")
final_df.head(10)

In [ ]:
# Records per year
final_df.groupby('ipl_year').size().reset_index(name='total_players')

In [ ]:
# Team distribution across all years
final_df['team_name'].value_counts()

In [ ]:
# Preview unsold players
final_df[final_df['team_name'] == 'Unsold'].head(10)

In [ ]:
# Save to Excel
final_df.to_excel('ipl_auction_all_years.xlsx', index=False)
print("Saved to ipl_auction_all_years.xlsx")